In [36]:
from os import path
import pandas as pd
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import FunctionTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
import pylab as py
from scipy.sparse import hstack
from scipy.sparse import coo_matrix
from tqdm import tqdm
from scipy import sparse
import csv, random, numpy, os, re, scipy, gensim
from sklearn.neural_network import MLPClassifier
from sklearn import tree
from langdetect import detect
from sklearn.ensemble import RandomForestClassifier
from csv import DictReader
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBClassifier

import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
# import score

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jimue\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jimue\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\jimue\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

tangina kinopya kolang itong mother fucker

In [37]:
bodies = pd.read_csv('./Data_Stance/train_bodies.csv')
stances = pd.read_csv('./Data_Stance/train_stances.csv')

dtset1_bodies = pd.DataFrame(bodies)
dtset1_stances = pd.DataFrame(stances)

class DataSet():
    def __init__(self, path="data"):
        self.path = path

        self.stances = self.read(stances)
        articles = self.read(bodies)
        self.articles = dict()

        #make the body ID an integer value
        for s in self.stances:
            s['Body ID'] = int(s['Body ID'])

        for article in articles:
            self.articles[int(article['Body ID'])] = article['articleBody']

        print("Total stances: " + str(len(self.stances)))
        print("Total bodies: " + str(len(self.articles)))

    def read(self,filename):
        rows = []
        with open( "Data_Stance/" + filename, "r", encoding='utf-8') as table:
            r = DictReader(table)

            for line in r:
                rows.append(line)
        return rows

In [40]:
pd.DataFrame.head(dtset1_bodies)

,Body ID,articleBody
0,0,A small meteorite crashed into a wooded area i...
1,4,Last week we hinted at what was to come as Ebo...
2,5,(NEWSER) – Wonder how long a Quarter Pounder w...
3,6,"Posting photos of a gun-toting child online, I..."
4,7,At least 25 suspected Boko Haram insurgents we...


In [41]:
pd.DataFrame.head(dtset1_stances)

,Headline,Body ID,Stance
0,Police find mass graves with at least '15 bodi...,712,unrelated
1,Hundreds of Palestinians flee floods in Gaza a...,158,agree
2,"Christian Bale passes on role of Steve Jobs, a...",137,unrelated
3,HBO and Apple in Talks for $15/Month Apple TV ...,1034,unrelated
4,Spider burrowed through tourist's stomach and ...,1923,disagree


In [42]:
dataset = DataSet()

Reading dataset
Total stances: 49972
Total bodies: 1683


In [43]:
dataset.stances[4]

{'Headline': "Spider burrowed through tourist's stomach and up into his chest",
 'Body ID': 1923,
 'Stance': 'disagree'}

In [31]:
def read_ids(file,base):
    ids = []
    with open(base+"/"+file,"r") as f:
        for line in f:
            ids.append(int(line))
        return ids
    
def split(dataset, base="splits"):
    if not (os.path.exists(base+"/"+"training_ids.txt")
            and os.path.exists(base+"/"+"dev_ids.txt") and os.path.exists(base+"/"+"test_ids.txt")):
        raise Exception("There is an error and the dataset reader cannot find the "
                        "{training_ids|test_ids|dev_ids}.txt file. Please make sure your python paths "
                        "are configured correctly")

    training_ids = read_ids("training_ids.txt",base)
    dev_ids = read_ids("dev_ids.txt",base)
    test_ids = read_ids("test_ids.txt",base)

    #return the stances that meet these criteria
    training_stances = []
    dev_stances = []
    test_stances = []

    for stance in dataset.stances:
        if stance['Body ID'] in training_ids:
            training_stances.append(stance)
        elif stance['Body ID'] in dev_ids:
            dev_stances.append(stance)
        elif stance['Body ID'] in test_ids:
            test_stances.append(stance)


    return {"training":training_stances, "dev":dev_stances, "test": test_stances}

In [32]:
data_splits = split(dataset)

training_data = data_splits['training']
dev_data = data_splits['dev']
test_data = data_splits['test']

N = int(len(training_data) * 1.0)
training_data = training_data[:N]

print("Training size:", len(training_data))
print("Dev size:", len(dev_data))
print("Test data:", len(test_data))

Training size: 40106
Dev size: 4835
Test data: 5031


In [44]:
len(dataset.articles)

1683

In [34]:
def get_bodies(data):
    bodies = []
    for i in range(len(data)):
        bodies.append(dataset.articles[data[i]['Body ID']])
    return bodies


def get_headlines(data):
    headlines = []
    for i in range(len(data)):
        headlines.append(data[i]['Headline'])
    return headlines

In [35]:
training_bodies = get_bodies(training_data)
training_headlines = get_headlines(training_data)
dev_bodies = get_bodies(dev_data)
dev_headlines = get_headlines(dev_data)
test_bodies = get_bodies(test_data)
test_headlines = get_headlines(test_data)

print(len(training_bodies))
print(len(training_headlines))
print(len(dev_bodies))
print(len(dev_headlines))
print(len(test_bodies))
print(len(test_headlines))

40106
40106
4835
4835
5031
5031
